In [1]:
#Libraries for files preparation

import os
import shutil
import pandas as pd
from sklearn.utils import shuffle

COVID-19 Radiography Database: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database

In [2]:
#Install: pip install openpyxl
covid = pd.read_excel('../COVID-19_Radiography_Dataset/COVID.metadata.xlsx')
covid.head()

,FILE NAME,FORMAT,SIZE,URL
0,COVID-1,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
1,COVID-2,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
2,COVID-3,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
3,COVID-4,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
4,COVID-5,PNG,256*256,https://sirm.org/category/senza-categoria/covi...


In [3]:
normal = pd.read_excel('../COVID-19_Radiography_Dataset/Normal.metadata.xlsx')
normal.head()

,FILE NAME,FORMAT,SIZE,URL
0,NORMAL-1,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
1,NORMAL-2,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
2,NORMAL-3,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
3,NORMAL-4,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
4,NORMAL-5,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...


In [4]:
#Checking the numbers of each class
print("COVID-19 has " +str(len(covid))+ " cases")
print("Normal has " +str(len(normal))+ " cases")

COVID-19 has 3616 cases
Normal has 10192 cases


In [5]:
SAMPLE_SIZE = 3616

In [6]:
#Add label for each case
normal['label'] = 0
covid['label'] = 1

In [7]:
#Drop non-related columns
normal = normal[['FILE NAME' ,'label']]
covid = covid[['FILE NAME', 'label']]

In [8]:
covid.head()

,FILE NAME,label
0,COVID-1,1
1,COVID-2,1
2,COVID-3,1
3,COVID-4,1
4,COVID-5,1


In [9]:
normal.head()

,FILE NAME,label
0,NORMAL-1,0
1,NORMAL-2,0
2,NORMAL-3,0
3,NORMAL-4,0
4,NORMAL-5,0


In [10]:
#Sampling data for covid and normal cases
df_normal = normal.sample(SAMPLE_SIZE, random_state=26)
df_covid = covid.sample(SAMPLE_SIZE, random_state=26)

#Concanate dataframes
data = pd.concat([df_normal, df_covid], axis=0)

#Checking numbers of each label
data['label'].value_counts()

0    3616
1    3616
Name: label, dtype: int64

In [11]:
#Shuflle data
data = shuffle(data)
data.head()

,FILE NAME,label
1816,NORMAL-1817,0
8868,NORMAL-8869,0
9010,NORMAL-9011,0
2791,COVID-2792,1
1230,NORMAL-1231,0


Train test split with the data (80:20)

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(data, test_size=0.2, random_state=26, stratify=data['label'])

print(df_train.shape)
print(df_val.shape)

(5785, 2)
(1447, 2)


In [13]:
df_train['label'].value_counts()

0    2893
1    2892
Name: label, dtype: int64

In [14]:
df_val['label'].value_counts()

1    724
0    723
Name: label, dtype: int64

Separating images into folders

In [15]:
#Creating a new directory to the image
data_dir = 'dataset'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    print(" The root dataset folder created!")

In [16]:
#Creating two folders inside 'data_dir'

#/dataset
#../train
#../../covid
#../../normal
#../val
#../../covid
#../../normal

#Creating a path to 'data_dir' wo which we will join the names of the new folders
#Creating folders inside the train and validation folders
#We create folders for each class, inside the each folder

#train
train_dir = os.path.join(data_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    print("\t-train folder created on dataset...")

#Creating new folders inside train_dir
train_covid = os.path.join(train_dir, 'covid')
if not os.path.exists(train_covid):
    os.mkdir(train_covid)
    print('\t\t-covid folder created on train folder...')

train_normal = os.path.join(train_dir, 'normal')
if not os.path.exists(train_normal):
    os.mkdir(train_normal)
    print('\t\t-normal folder created on train folder...')


#val
val_dir = os.path.join(data_dir, 'val')
if not os.path.exists(val_dir):
    os.mkdir(val_dir)
    print("\t-val folder created on dataset...")

#Creating new folders inside val_dir
val_covid = os.path.join(val_dir, 'covid')
if not os.path.exists(val_covid):
    os.mkdir(val_covid)
    print('\t\t-covid folder created on val folder...')

val_normal = os.path.join(val_dir, 'normal')
if not os.path.exists(val_normal):
    os.mkdir(val_normal)
    print('\t\t-normal folder created on val folder...')

In [17]:
#Check the folders in train_dir
os.listdir(train_dir)

['covid', 'normal']

Transfer the images into folders

In [18]:
train_list = list(df_train['FILE NAME'])
val_list = list(df_val['FILE NAME'])

In [19]:
#Copying images to train_dir folder

for image in train_list:
    filename = image + '.png'
    #Get the label for a certain image
    target = int(data.loc[data['FILE NAME'] == image, ['label']].values)
    #Match the target with the folder's name and source path of the image
    if target == 1:
        label = 'covid'
        src = os.path.join('../COVID-19_Radiography_Dataset/COVID/images', filename)

    elif target == 0:
        label = 'normal'
        filename = filename.capitalize()
        src = os.path.join('../COVID-19_Radiography_Dataset/Normal/images', filename)

    
    dest = os.path.join(train_dir, label, filename)
    shutil.copyfile(src=src, dst=dest)

In [20]:
#Copying images to val_dir folder

for image in val_list:
    filename = image + '.png'
    target = int(data.loc[data['FILE NAME'] == image, ['label']].values)

    if target == 1:
        label = 'covid'
        src = os.path.join('../COVID-19_Radiography_Dataset/COVID/images', filename)

    elif target == 0:
        label = 'normal'
        filename = filename.capitalize()
        src = os.path.join('../COVID-19_Radiography_Dataset/Normal/images', filename)
    

    dest = os.path.join(val_dir, label, filename)
    shutil.copyfile(src=src, dst=dest)

In [21]:
print('Train - COVID: ',len(os.listdir('dataset/train/covid')))
print('Train - NORMAL: ',len(os.listdir('dataset/train/normal')))
print('Validation - COVID: ',len(os.listdir('dataset/val/covid')))
print('Validation - NORMAL: ',len(os.listdir('dataset/val/normal')))

Train - COVID:  3485
Train - NORMAL:  3474
Validation - COVID:  1317
Validation - NORMAL:  1304
